## Goals: Training the *Final* Models

This notebook trains the model on the full *baseline_dataset* for the final prediction on evaluation data.

Here, we train a model designed to generalize across water stations in Brazil and France. However, you are not required to follow this approach and may opt to train separate models for different geographic *regions*.

This baseline model training example utilizes all available features, with hyperparameters chosen for quick execution rather than optimization. For hyperparameter tuning and feature selection explorations, refer to the `02_exploration` folder.

> **Note:** This notebook requires outputs from the `00 Preprocessing` notebooks.

<img src="../images/notebook-3.png" alt="Experiment Diagram" style="width:75%; text-align:center;" />

### 1. Data Import and Setup

This section imports the necessary libraries, sets up environment paths, and includes custom utility functions.

In [1]:
import os
import sys

import joblib
import numpy as np
import pandas as pd
import lightgbm as lgb

from quantile_forest import RandomForestQuantileRegressor
from mapie.regression import MapieQuantileRegressor
from interpret.glassbox import ExplainableBoostingRegressor

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..', '..')))

from src.utils.model import split_dataset, compare_models_per_station

##### Constants :
- **INPUT_DIR**: Directory for input data (same as in "02 - Feature Engineering").
- **MODEL_DIR**: Directory where trained models are saved.
- **DATASET_DIR**: Directory where the Zenodo dataset is unzipped.

##### Model Parameters

- **SEED**: 42 (for reproducibility)
- **NUMBER_OF_WEEK**: 4 (one model is trained per week)

##### FINAL_MODELS

- **mapie**: Combines LightGBM with MAPIE. **MAPIE** (Model Agnostic Prediction Interval Estimator) computes prediction intervals for any regression model using conformal methods.
- **qrf**: Quantile Random Forest (natively produces prediction intervals)
- **ebm**: Explainable Boosting Machine is used as a exemple that does not natively implement prediction intervals, but that can be customised to do so.

In [2]:
INPUT_DIR = "../../../data/input/"
MODEL_DIR = "../../../models/"
DATASET_DIR = "../../../dataset/"

SEED = 42
NUMBER_OF_WEEK = 4 # Number of weeks to predict one model is trained per week

FINAL_MODELS = ["mapie",
                "qrf",
                #"EBM"
                ]
mapie_enbpi = {}
mapie = {}
qrf = {}
mapie_aci = {}

COLUMNS_TO_DROP = ["water_flow_week1", "station_code", "water_flow_week2", "water_flow_week3", "water_flow_week4"]


### 2. Data Loading
Load in the baseline datasets, create the directory to save models.

In [3]:
dataset_train = pd.read_csv(f"{INPUT_DIR}dataset_baseline.csv")

dataset_train = dataset_train.set_index("ObsDate")

if not os.path.exists(f"{MODEL_DIR}final/"):
    os.makedirs(f"{MODEL_DIR}final/")

Data pre-processing removal of unnecessary columns, setup of the target

In [4]:
X_train = dataset_train.drop(columns=COLUMNS_TO_DROP)
y_train = {}
for i in range(0, NUMBER_OF_WEEK):
    y_train[i] = dataset_train[f"water_flow_week{i+1}"]


### 2. Models training
#### a. LGBM + MAPIE

## Mapie Model Training Overview

- **Configuration:**  
  - Sets `ALPHA` (0.1) as the prediction interval level.
  - Defines `TIME_VALIDATION` as a split point for creating a validation set.
  - Configures LightGBM parameters (`LGBM_PARAMS`) for quantile regression.




In [5]:
ALPHA = 0.1
TIME_VALIDATION = "2000-01-01 00:00:00"

LGBM_PARAMS = {
    "max_depth": 15,
    "learning_rate": 0.01,
    "n_estimators": 500,
    "colsample_bytree": 0.7,
    "objective": "quantile",
    "alpha": ALPHA
}

- **Data Preparation:**  
  - Splits `dataset_train` into training and validation subsets using `split_dataset`.
  - Removes unnecessary columns from both the training and validation datasets.
  - Extracts target variables for each week (from `water_flow_week1` to `water_flow_week4`).

- **Model Training:**  
  For each week:
  - Initializes a LightGBM regressor with the specified parameters.
  - Wraps it in a `MapieQuantileRegressor` to estimate prediction intervals.
  - Trains the model on the training data and calibrates it using the validation data.
  - Saves the trained model 

In [6]:
if "mapie" in FINAL_MODELS: 
    print("Training Mapie")


    train_mapie, val_mapie, val_temporal  = split_dataset(dataset_train, 0.75, TIME_VALIDATION)

    X_train_mapie = train_mapie.drop(columns=COLUMNS_TO_DROP)
    print(len(X_train_mapie.columns))
    y_train_mapie = {}
    for i in range(0, NUMBER_OF_WEEK):
        y_train_mapie[i] = train_mapie[f"water_flow_week{i+1}"]

    X_val = val_mapie.drop(columns=COLUMNS_TO_DROP)
    y_val = {}
    y_val[0] = val_mapie["water_flow_week1"]
    for i in range(1, NUMBER_OF_WEEK):
        y_val[i] = val_mapie[f"water_flow_week{i+1}"]

    for i in range(NUMBER_OF_WEEK):
        print(f"Training week {i}")
        # Initialize and train MapieQuantileRegressor
        regressor = lgb.LGBMRegressor(**LGBM_PARAMS)
        mapie[i] = MapieQuantileRegressor(estimator=regressor, method="quantile", cv="split", alpha=ALPHA)
        mapie[i].fit(X_train_mapie, y_train_mapie[i], X_calib=X_val, y_calib=y_val[i])
        
        # save model with date
        time = pd.Timestamp.now().strftime("%Y-%m-%d_%H-%M-%S")

        model_path = f"{MODEL_DIR}final/mapie_quantile_{time}_week_{i}.pkl"
        joblib.dump(mapie[i], model_path)


Training Mapie
154
Training week 0
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001701 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 11458
[LightGBM] [Info] Number of data points in the train set: 15051, number of used features: 154
[LightGBM] [Info] Start training from score 0.218643
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008569 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 11458
[LightGBM] [Info] Number of data points in the train set: 15051, number of used features: 154
[LightGBM] [Info] Start training from score 541.075745
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002580 seconds.
You can set `force_row_wise=true` to remove

#### b. QRF

- **Training:**  
  Initializes a `RandomForestQuantileRegressor` with the following parameters:
  - 100 estimators
  - Maximum depth of 10
  - Minimum of 10 samples per leaf

  These parameters allow for relatively fast training, though they are not optimized for peak performance. 
  
  The model is then fitted using `X_train` and the corresponding weekly target `y_train[i]`.

In [7]:
if "qrf" in FINAL_MODELS:
    for i in range(NUMBER_OF_WEEK):
        print(f"Training week {i}")
        # Train RandomForestQuantileRegressor
        qrf[i] = RandomForestQuantileRegressor(n_estimators=100, max_depth=5, min_samples_leaf=5)
        qrf[i].fit(X_train, y_train[i])

        time = pd.Timestamp.now().strftime("%Y-%m-%d_%H-%M-%S")
        model_path = f"{MODEL_DIR}final/qrf_quantile_{time}_week_{i}.pkl"
        joblib.dump(qrf[i], model_path)

Training week 0
Training week 1
Training week 2
Training week 3


#### c. Explainable Boosting Machine

EBM is an ensemble method that does not natively provide access to its individual members for performing quantile predictions or generating prediction intervals. To overcome this limitation, we manually construct an ensemble.


- **Ensemble Training:**  
- For each ensemble member (seed from 0 to 4):
    - A bootstrap sample is created from `X_train` and `y_train[i]` using sampling with replacement.
    - An `ExplainableBoostingRegressor` is instantiated with fixed parameters (e.g., `max_bins=128`, `learning_rate=0.05`, `interactions=3`, and `random_state=42` to ensure consistent binning) and then trained on the sampled data.
    - The trained model is appended to the list for the current week.
- **Saving the Ensemble:**  
- The ensemble (i.e., the list of EBM models for the week) is saved.

In [8]:
if "ebm" in FINAL_MODELS:
    NUM_ENSEMBLES = 5
    ebm_ensembles = {}
    for i in range(NUMBER_OF_WEEK):
        print(f"Training EBM ensemble for week {i}")

        models_i = []
        for seed in range(NUM_ENSEMBLES):
            print(f"Training EBM ensemble {seed} for week {i}")
            # 1. Create your bootstrap sample or subset (if you want bagging)
            sample_indices = np.random.choice(len(X_train), size=len(X_train), replace=True)
            X_sample = X_train.iloc[sample_indices]
            y_sample = y_train[i][sample_indices]
            
            # 2. Train an EBM with consistent binning parameters
            ebm_model = ExplainableBoostingRegressor(
                outer_bags=1,
                inner_bags=1,
                max_bins=128,
                learning_rate=0.05,
                interactions=3,
                early_stopping_rounds=100,
                random_state=SEED
            )
            ebm_model.fit(X_sample, y_sample)
            
            models_i.append(ebm_model)

        time = pd.Timestamp.now().strftime("%Y-%m-%d_%H-%M-%S")
        file_path = f"{MODEL_DIR}final/ebm_ensemble_{time}_week_{i}.pkl"

        joblib.dump(ebm_ensembles, file_path)
        print(f"Saved EBM ensembles to {file_path}")

        # Store the list of models for week i
        ebm_ensembles[i] = models_i

#### d. Bayesian Structural Time Series with Variational Inference method

Note: This section is used for training and prediction for Bayesian method. For other method refer to the "Prediction Computation" notebook for prediction.

In [ ]:
import random
import zipfile

from scipy.stats import iqr
import tensorflow as tf
import tensorflow_probability as tfp

In [ ]:
tf.random.set_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

In [ ]:
EVAL_DIR = "../../../data/evaluation/"
COLUMNS_TO_DROP_STS = ["water_flow_week1", "water_flow_week2", "water_flow_week3", "water_flow_week4"]

In [ ]:
# Read train/test datasets
dataset_train = pd.read_csv(f"{INPUT_DIR}dataset_baseline.csv")
dataset_test = pd.read_csv(f"{EVAL_DIR}dataset_baseline.csv")
print(dataset_train.shape, dataset_test.shape)

(28431, 160) (1352, 156)


In [ ]:
def compute_station_stats(df, station_col, value_col):
    return df.groupby(station_col)[value_col].agg([
        "median",
        ("iqr", lambda x: iqr(x))
    ]).to_dict(orient="index")

def normalize_station(df, y_column):
    median = df[y_column].median()
    iqr_value = iqr(df[y_column])
    df[f"{y_column}_normalized"] = (df[y_column] - median) / iqr_value # Robust Scaling (IQR Scaling)
    return df

In [ ]:
station_stats = {}
for column in COLUMNS_TO_DROP_STS:
    station_stats[column] = compute_station_stats(dataset_train, "station_code", column)
    dataset_train = dataset_train.groupby("station_code").apply(lambda df: normalize_station(df, column)).reset_index(drop=True)
# dataset_train = dataset_train.set_index(["ObsDate", "station_code"])
dataset_train.shape

/var/folders/m6/rpntmn7d1j3319llfgc9lhq40000gn/T/ipykernel_4271/4015224528.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  dataset_train = dataset_train.groupby("station_code").apply(lambda df: normalize_station(df, column)).reset_index(drop=True)
/var/folders/m6/rpntmn7d1j3319llfgc9lhq40000gn/T/ipykernel_4271/4015224528.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  dataset_train = dataset_train.g

(28431, 164)

In [ ]:
dataset_full = pd.concat([dataset_train, dataset_test])
dataset_full = dataset_full.set_index(["ObsDate", "station_code"])
dataset_full.shape

(29783, 162)

In [ ]:
COLUMNS_TO_DROP_BSTS_NORM = COLUMNS_TO_DROP_STS+[f"{col}_normalized" for col in COLUMNS_TO_DROP_STS]
X_train = dataset_full.drop(columns=COLUMNS_TO_DROP_BSTS_NORM)
y_train = {}
for i in range(0, NUMBER_OF_WEEK):
    y_train[i] = dataset_full[f"water_flow_week{i+1}_normalized"]

In [ ]:
# Convert categorical station IDs to indices
station_codes = X_train.reset_index()['station_code'].apply(lambda x: str(x)[:3]).astype("category").cat.codes.values.astype(np.int32)
num_stations = X_train.reset_index()['station_code'].apply(lambda x: str(x)[:3]).nunique()

# Define station embeddings (learnable station-specific parameters)
station_embedding_dim = 3  # Adjust as needed
station_embeddings = tf.Variable(tf.random.normal([num_stations, station_embedding_dim]), name="station_embeddings")

# Station embeddings as hierarchical factors
station_inputs = tf.gather(station_embeddings, station_codes)  # Get station-specific embeddings

In [ ]:
def build_bsts_model(observed_time_series):
  features_effect = tfp.sts.SparseLinearRegression(
      design_matrix=np.hstack([X_train.values, station_inputs.numpy()]).astype(np.float32), name='features_effect')
  observation_noise_prior = tfp.distributions.LogNormal(loc=0.0, scale=1.0)  # Prior with higher variance

  model = tfp.sts.Sum([features_effect],
                      observed_time_series=observed_time_series,
                      observation_noise_scale_prior=observation_noise_prior
                    )
  return model


def train_bayesian_sts(y_training_data, model):
    # Build the variational surrogate posteriors `qs`.
    variational_posteriors = tfp.sts.build_factored_surrogate_posterior(model=model, seed=SEED)
    num_variational_steps = 200 
    # Build and optimize the variational loss function.
    elbo_loss_curve = tfp.vi.fit_surrogate_posterior(
        target_log_prob_fn=model.joint_distribution(
            observed_time_series=y_training_data).log_prob,
        surrogate_posterior=variational_posteriors,
        optimizer=tf.optimizers.Adam(learning_rate=0.1),
        num_steps=num_variational_steps,
        jit_compile=True,
        seed=SEED
    )
    
    return elbo_loss_curve, variational_posteriors


def aggregate_by_prefix(data, prefix_length):
    grouped_data = {}

    for key, stats in data.items():
        prefix = str(key)[:prefix_length]  # Extract first 3 digits if prefix=3
        if prefix not in grouped_data:
            grouped_data[prefix] = {"median": [], "iqr": []}
        grouped_data[prefix]["median"].append(stats["median"])
        grouped_data[prefix]["iqr"].append(stats["iqr"])

    # Calculate the mean of the medians and IQRs for each group
    aggregated_stats = {
        prefix: {
            "median": np.mean(values["median"]),
            "iqr": np.mean(values["iqr"])
        }
        for prefix, values in grouped_data.items()
    }

    return aggregated_stats


def denormalize_predictions(df, y_pred_normalized, column_name):
    y_pred_real = []
    station_stats_prefix3 = aggregate_by_prefix(station_stats[column_name], 3)
    station_stats_prefix2 = aggregate_by_prefix(station_stats[column_name], 2)
    station_stats_prefix1 = aggregate_by_prefix(station_stats[column_name], 1)
    for station, y_pred in zip(df["station_code"], y_pred_normalized):
        if station in station_stats[column_name].keys():
            median, iqr_value = station_stats[column_name][station].values()  # Retrieve station stats

        elif str(station)[:3] in station_stats_prefix3.keys():
            median, iqr_value = station_stats_prefix3[str(station)[:3]].values()

        elif str(station)[:2] in station_stats_prefix2.keys():
            median, iqr_value = station_stats_prefix2[str(station)[:2]].values()

        elif str(station)[:1] in station_stats_prefix1.keys():
            median, iqr_value = station_stats_prefix1[str(station)[:1]].values()

        reverse_norm = (y_pred * iqr_value) + median  # Denormalization
        y_pred_real.append(reverse_norm)  
    
    return y_pred_real


In [ ]:
### ----- Training section ----- ###

num_forecast_steps = 1352 # 6 years.
n_sample = 500
for index, y_train_week in y_train.items():
    print(f"Training week {index}")
    y_training_data = y_train_week[:-num_forecast_steps].values.astype(np.float32)
    model = build_bsts_model(y_training_data)
    _, variational_posteriors = train_bayesian_sts(y_training_data, model)
    # Draw samples from the variational posterior.
    q_samples = variational_posteriors.sample(n_sample)
    # Sauvegarder avec TensorFlow SavedModel
    # time = pd.Timestamp.now().strftime("%Y-%m-%d_%H-%M-%S")
    posterior_path = f"{MODEL_DIR}final/bsts_vi_posterior_samples_week_{index}.npy"
    np.save(posterior_path, q_samples)

Training week 0


I0000 00:00:1741190407.338305 1302870 service.cc:148] XLA service 0x11a3bf320 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1741190407.338686 1302870 service.cc:156]   StreamExecutor device (0): Host, Default Version
2025-03-05 17:00:07.492662: W tensorflow/compiler/tf2xla/kernels/random_ops.cc:107] Warning: Using tf.random.uniform with XLA compilation will ignore seeds; consider using tf.random.stateless_uniform instead if reproducible behavior is desired. fit_surrogate_posterior/sanitize_seed/seed
2025-03-05 17:00:07.522152: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1741190411.593498 1302870 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Training week 1
Training week 2
Training week 3


In [ ]:
### ----- Forecast section ----- ###

forecast_df = pd.DataFrame({
    "ObsDate": dataset_test['ObsDate'].values,
    "station_code": dataset_test['station_code'].values,
})
for index, y_train_week in y_train.items():
    y_training_data = y_train_week[:-num_forecast_steps].values.astype(np.float32)
    model = build_bsts_model(y_training_data)
    # Load the saved posterior samples:
    loaded_posterior_samples = np.load(f"{MODEL_DIR}final/bsts_vi_posterior_samples_week_{index}.npy", allow_pickle=True).item()

    # Generate forecast samples
    forecast_dist = tfp.sts.forecast(
        model=model,
        observed_time_series=y_training_data,
        parameter_samples=loaded_posterior_samples,
        num_steps_forecast=num_forecast_steps)
    mean_forecast = forecast_dist.mean().numpy()
    forecast_samples = forecast_dist.sample(n_sample).numpy()
    # Compute 70% credible interval (percentiles 15% and 85%) for each forecast
    forecast_lower = np.percentile(forecast_samples, 30, axis=0)
    forecast_upper = np.percentile(forecast_samples, 70, axis=0)
    # Denormalisation
    mean_forecast_denorm = denormalize_predictions(forecast_df, mean_forecast.flatten(), f"water_flow_week{index+1}")
    forecast_lower_denorm = denormalize_predictions(forecast_df, forecast_lower.flatten(), f"water_flow_week{index+1}")
    forecast_upper_denorm = denormalize_predictions(forecast_df, forecast_upper.flatten(), f"water_flow_week{index+1}")
    forecast_df[f"week_{index}_pred"] = mean_forecast_denorm
    forecast_df[f"week_{index}_inf"] = forecast_lower_denorm
    forecast_df[f"week_{index}_sup"] = forecast_upper_denorm
    

In [ ]:
# Save forecast series
forecast_df.to_csv(f"{EVAL_DIR}/predictions_bsts.csv", index=False)
# Create a ZIP file containing predictions.csv
with zipfile.ZipFile(f"{EVAL_DIR}predictions_bsts.zip", 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipf.write(f"{EVAL_DIR}predictions_bsts.csv", "predictions_bsts.csv")

### 3. Performance Evaluation on the Full Training Set

> **Note:**  
> The performance displayed here is calculated on the training set. This does not necessarily reflect the models' performance on unseen data.


In [ ]:
y_train_stations = dataset_train["station_code"].values

for i in range(NUMBER_OF_WEEK):
    predictions = []
    baseline_day_before = dataset_train["water_flow_lag_1w"]
    predictions.append({"model": "Week before", "prediction": baseline_day_before, "dataset":"train", "stations": y_train_stations, "prediction_interval": None})
    if "mapie" in FINAL_MODELS:
        y_pred_mapie, y_pis_mapie = mapie[i].predict(X_train)
        predictions.append({"model": "LGBM+MAPIE", "prediction": y_pred_mapie, "dataset":"train", "stations": y_train_stations, "prediction_interval": y_pis_mapie})
    if "qrf" in FINAL_MODELS:
        y_pred_qrf = qrf[i].predict(X_train, quantiles="mean", aggregate_leaves_first=False)
        y_pis_qrf = qrf[i].predict(X_train, quantiles=[ALPHA/2, 1-ALPHA/2])
        predictions.append({"model": "QRF", "prediction": y_pred_qrf, "dataset":"train", "stations": y_train_stations, "prediction_interval": y_pis_qrf})
    if "ebm" in FINAL_MODELS:
        y_pred_ebm = []
        for model in ebm_ensembles[i]:
            y_pred_ebm.append(model.predict(X_train))
        y_pred_ebm = np.mean(y_pred_ebm, axis=0)
        predictions.append({"model": "EBM", "prediction": y_pred_ebm, "dataset":"train", "stations": y_train_stations, "prediction_interval": None})

    compare_models_per_station(
        y_train[i].values,
        predictions,
        y_train_stations,
        column_to_display="log_likelihood" ,
        title = f"WEEK {i}")

### 4. Coverage on the Full Training Set

> **Note:**  
> The performance displayed here is calculated on the training set. This does not necessarily reflect the models' performance on unseen data.


In [ ]:
for i in range(NUMBER_OF_WEEK):

    predictions = []
    baseline_day_before = dataset_train["water_flow_lag_1w"]
    predictions.append({"model": "Week before", "prediction": baseline_day_before, "dataset":"train", "stations": y_train_stations, "prediction_interval": None})
    if "mapie" in FINAL_MODELS:
        y_pred_mapie, y_pis_mapie = mapie[i].predict(X_train)
        predictions.append({"model": "LGBM+MAPIE", "prediction": y_pred_mapie, "dataset":"train", "stations": y_train_stations, "prediction_interval": y_pis_mapie})
        coverage = (y_train[i].values >= y_pis_mapie[:,0,0]) & (y_train[i].values <= y_pis_mapie[:,1,0])
        print(f"MAPIE coverage of the prediction interval for week {i}: {coverage.mean()}")
    if "qrf" in FINAL_MODELS:
        y_pred_qrf = qrf[i].predict(X_train, quantiles="mean", aggregate_leaves_first=False)
        y_pis_qrf = qrf[i].predict(X_train, quantiles=[ALPHA/2, 1-ALPHA/2])
        predictions.append({"model": "QRF", "prediction": y_pred_qrf, "dataset":"train", "stations": y_train_stations, "prediction_interval": y_pis_qrf})
        coverage = (y_train[i].values >= y_pis_qrf[:,0]) & (y_train[i].values <= y_pis_qrf[:,1])
        print(f"QRF coverage of the prediction interval for week {i}: {coverage.mean()}")
    if "ebm" in FINAL_MODELS:
        y_pred_ebm = []
        for model in ebm_ensembles[i]:
            y_pred_ebm.append(model.predict(X_train))
        y_pred_ebm = np.mean(y_pred_ebm, axis=0)
        predictions.append({"model": "EBM", "prediction": y_pred_ebm, "dataset":"train", "stations": y_train_stations, "prediction_interval": None})
        coverage = (y_train[i].values >= y_pis_qrf[:,0]) & (y_train[i].values <= y_pis_qrf[:,1])
        print(f"EBM coverage of the prediction interval for week {i}: {coverage.mean()}")